# Multiple Mercury

Simulate a Solar System, in a *jumping jupiter* scenario, evaluating different configurations of semi-axes for a hypothetical planet Mercury in a compact orbit.

Tasks:
+ Create directory tree
+ Create the Swift configuration files
+ Run the simulation
+ Extract figures


## Create directory tree

In [118]:
import os
import shutil
from glob import glob
import pandas as pd
import numpy as np
import math

import sys
sys.path.insert(0, 'src/')
import oe2pv

In [119]:
simulation_name = "mercury"
level1 = 4 # Number of mercury varying with distance 
level2 = 5 # Number of clones
simulation_path = "output/" + simulation_name
absolute_path = os.getcwd()

In [120]:
os.chdir("output")

if os.path.isdir(simulation_name):
    shutil.rmtree(simulation_name)
    os.mkdir(simulation_name)
    os.chdir(simulation_name)
else:
    os.mkdir(simulation_name)
    os.chdir(simulation_name)

for i in range(level1):
    for j in range(level2):
        os.mkdir(simulation_name + "-" + "{:03d}".format(i) + "-" + "{:03d}".format(j))
        
os.chdir(absolute_path)

### Check directories tree

In [4]:
!ls output/mercury/

mercury-000-000  mercury-001-000  mercury-002-000  mercury-003-000
mercury-000-001  mercury-001-001  mercury-002-001  mercury-003-001
mercury-000-002  mercury-001-002  mercury-002-002  mercury-003-002
mercury-000-003  mercury-001-003  mercury-002-003  mercury-003-003
mercury-000-004  mercury-001-004  mercury-002-004  mercury-003-004


## Create the Swift configuration files

### Create planets dataframe

In [73]:
# Create planets data (J2000)
# 
# Data from https://nssdc.gsfc.nasa.gov/planetary/
# Data sequence: 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus',
#  'Neptune'.

# Planet's names
planets_name = pd.Series(['mercury','venus', 'earth', 'mars', 'jupiter', \
                          'saturn', 'uranus', 'neptune'])

# Semi-axis [au]
a = pd.Series([0.38709893, 0.72333199, 1.00000011, 1.52366231, 5.20336301,\
               9.53707032, 19.19126393, 30.06896348])

e = pd.Series([0.205, 0.00677323, 0.01671022, 0.09341233, 0.04839266, 0.0541506,\
               0.04716771, 0.00858587])

i = pd.Series([7.0, 3.39471, 5e-05, 1.85061, 1.3053, 2.48446, 0.76986, 1.76917])

capom = pd.Series([1.0, 76.68069, -11.26064, 49.57854, 100.55615, 113.71504, \
                   74.22988, 131.72169])

omega = pd.Series([90.0, 131.53298, 102.94719, 336.04084, 14.75385, 92.43194, \
                   170.96424, 44.97135])


# Mass [kg]
mass = pd.Series([0.33011, 4.8675, 5.9723, 0.64171, 1898.19, 568.34, 86.813,\
                  102.413]) * 1e24

# Equatorial radius [km]
radio = pd.Series([2439.7, 6051.8, 6378.137, 3396.2, 71492, 60268, 25559, 24764])

# Period [days]
period = pd.Series([87.969, 224.701, 365.256, 686.980, 4332.589, 10759.22,\
                    30685.4, 60189])

M = pd.Series([90.0, 181.97973, 100.46435, 355.45332, 34.40438, 49.94432, 313.23218, \
               304.88003])


# Create a data frame from data series
planets = pd.DataFrame({'planets_name': planets_name, 'a': a, 'e': e, 'i': i,\
                        'capom': capom, 'omega': omega,'mass': mass, \
                        'radio': radio, 'period': period, 'M': M})

# Make planets_name index
#planets = planets.set_index('planets_name')

In [6]:
#Create new column, considering G = 1
# Mass of the Sum, in kg
mass_sun_kg = 1988500e24

# Mass of the Sun, with G = 1
mass_sun_grav = 2.959139768995959e-04

# Conic section is ellipse
ialpha = -1

# Gravitational factor of the Sun
gm =  2.959139768995959e-04

# Create mass_grav column
planets['mass_grav'] = planets.mass * mass_sun_grav / mass_sun_kg

# Create gmpl
planets['gmpl'] = gm + planets.mass_grav

In [7]:
# Que coisa linda!!!
for i in planets_name:
    exec(i + ' = planets.loc[planets.planets_name == i].drop("planets_name", 1)')

In [259]:
for i in planets_name:
    print(i)

mercury
venus
earth
mars
jupiter
saturn
uranus
neptune


### Check planets' dataframe

In [8]:
print("\nMercury\n", mercury)
print("\nEarth\n", earth)
print("\nJupiter\n", jupiter)


Mercury
       M         a  capom      e    i          mass  omega  period   radio  \
0  90.0  0.387099    1.0  0.205  7.0  3.301100e+23   90.0  87.969  2439.7   

      mass_grav      gmpl  
0  4.912455e-11  0.000296  

Earth
            M    a     capom        e        i          mass      omega  \
2  100.46435  1.0 -11.26064  0.01671  0.00005  5.972300e+24  102.94719   

    period     radio     mass_grav      gmpl  
2  365.256  6378.137  8.887539e-10  0.000296  

Jupiter
           M         a      capom         e       i          mass     omega  \
4  34.40438  5.203363  100.55615  0.048393  1.3053  1.898190e+27  14.75385   

     period    radio     mass_grav      gmpl  
4  4332.589  71492.0  2.824747e-07  0.000296  


### Test to verify Mercury's inclination in a especific date.

### Include angle for each planet

In [57]:
def angles_for_planets(pl):
    # Creating variables to use the "orbel" function
    gm = pl['gmpl']
    a = pl['a']
    e = pl['e']
    inc = pl['i']
    capom = pl['capom']
    omega = pl['omega']
    capm = pl['M']
    P = pl['period']
    rpl = pl['radio']

    #Create position and velocity columns
    #
    # The module eo2pv.so, constructed from the Swift conversion subroutine,
    # will be used.

    x = np.zeros(len(pl))
    y = np.zeros(len(pl))
    z = np.zeros(len(pl))
    vx = np.zeros(len(pl))
    vy = np.zeros(len(pl))
    vz = np.zeros(len(pl))

    for j in range(len(pl)):
        x[j], y[j], z[j], vx[j], vy[j], vz[j] = oe2pv.orbel_el2xv(gm[j], ialpha, a[j],e[j],\
                                                            math.radians(inc[j]),\
                                                            math.radians(capom[j]),\
                                                            math.radians(omega[j]),\
                                                            capm[j])


#     # Create colums x, y, v, vx, vy and vz
#     pl['x'] = x
#     pl['y'] = y
#     pl['z'] = z
#     pl['vx'] = vx
#     pl['vy'] = vy
#     pl['vz'] = vz

    return x, y, z, vx, vy, vz

In [58]:
for j in range(len(pl)):
    x[j], y[j], z[j], vx[j], vy[j], vz[j] = oe2pv.orbel_el2xv([gmpl][j], ialpha, a[j],e[j],\
                                                        math.radians(inc[j]),\
                                                        math.radians(capom[j]),\
                                                        math.radians(omega[j]),\
                                                        capm[j])

In [61]:
func_vec = np.vectorize(oe2pv.orbel_el2xv)

In [ ]:
for i in planets_name:
    x, y, z, vx, vy, vz = oe2pv.orbel_el2xv(i["gmpl"][j], ialpha, a[j],e[j],\
                                                        math.radians(inc[j]),\
                                                        math.radians(capom[j]),\
                                                        math.radians(omega[j]),\
                                                        capm[j])

In [10]:
os.chdir(absolute_path)
os.chdir("output/" + simulation_name)

simulations = glob("*")
for simulation in simulations:
    os.chdir(simulation)
    with open("pl.in", "w") as f:
        f.write("123\n456\n789")
    os.chdir("../")

os.chdir(absolute_path)

## Run the simulation

## Extract figures

## Teste com arrays

In [67]:
mars.values

array([[  3.55453320e+02,   1.52366231e+00,   4.95785400e+01,
          9.34123300e-02,   1.85061000e+00,   6.41710000e+23,
          3.36040840e+02,   6.86980000e+02,   3.39620000e+03,
          9.54945729e-11,   2.95914072e-04]])

In [142]:
mercury = planets[["a", "e", "i", "capom", "omega", "M", "radio", "period", "mass"]].iloc[0].values
venus = planets[["a", "e", "i", "capom", "omega", "M", "radio", "period", "mass"]].iloc[1].values
earth = planets[["a", "e", "i", "capom", "omega", "M", "radio", "period", "mass"]].iloc[2].values
mars = planets[["a", "e", "i", "capom", "omega", "M", "radio", "period", "mass"]].iloc[3].values
jupiter = planets[["a", "e", "i", "capom", "omega", "M", "radio", "period", "mass"]].iloc[4].values
saturn = planets[["a", "e", "i", "capom", "omega", "M", "radio", "period", "mass"]].iloc[5].values
uranus = planets[["a", "e", "i", "capom", "omega", "M", "radio", "period", "mass"]].iloc[6].values
neptune = planets[["a", "e", "i", "capom", "omega", "M", "radio", "period", "mass"]].iloc[7].values

In [143]:
print("mercury = np.array(", list(mercury), ")")

print("venus = np.array(", list(venus), ")")

print("earth = np.array(", list(earth), ")")

print("mars = np.array(", list(mars), ")")

print("jupiter = np.array(", list(jupiter), ")")

print("saturn = np.array(", list(saturn), ")")

print("uranus = np.array(", list(uranus), ")")

print("neptune = np.array(", list(neptune), ")")

mercury = np.array( [0.38709893000000001, 0.20499999999999999, 7.0, 1.0, 90.0, 90.0, 2439.6999999999998, 87.968999999999994, 3.3011000000000001e+23] )
venus = np.array( [0.72333199000000004, 0.0067732299999999999, 3.3947099999999999, 76.680689999999998, 131.53298000000001, 181.97972999999999, 6051.8000000000002, 224.70099999999999, 4.8674999999999993e+24] )
earth = np.array( [1.00000011, 0.016710220000000001, 5.0000000000000002e-05, -11.26064, 102.94719000000001, 100.46435, 6378.1369999999997, 365.25599999999997, 5.9722999999999996e+24] )
mars = np.array( [1.52366231, 0.093412330000000002, 1.8506100000000001, 49.578539999999997, 336.04084, 355.45332000000002, 3396.1999999999998, 686.98000000000002, 6.4171000000000003e+23] )
jupiter = np.array( [5.2033630100000003, 0.048392659999999997, 1.3052999999999999, 100.55615, 14.75385, 34.404380000000003, 71492.0, 4332.5889999999999, 1.8981900000000001e+27] )
saturn = np.array( [9.5370703199999998, 0.0541506, 2.4844599999999999, 113.71504, 92.43

In [167]:
mercury = np.array( [0.38709893000000001, 0.20499999999999999, 7.0, 1.0, 90.0, 90.0, 2439.6999999999998, 87.968999999999994, 3.3011000000000001e+23] )
venus = np.array( [0.72333199000000004, 0.0067732299999999999, 3.3947099999999999, 76.680689999999998, 131.53298000000001, 181.97972999999999, 6051.8000000000002, 224.70099999999999, 4.8674999999999993e+24] )
earth = np.array( [1.00000011, 0.016710220000000001, 5.0000000000000002e-05, -11.26064, 102.94719000000001, 100.46435, 6378.1369999999997, 365.25599999999997, 5.9722999999999996e+24] )
mars = np.array( [1.52366231, 0.093412330000000002, 1.8506100000000001, 49.578539999999997, 336.04084, 355.45332000000002, 3396.1999999999998, 686.98000000000002, 6.4171000000000003e+23] )
jupiter = np.array( [5.2033630100000003, 0.048392659999999997, 1.3052999999999999, 100.55615, 14.75385, 34.404380000000003, 71492.0, 4332.5889999999999, 1.8981900000000001e+27] )
saturn = np.array( [9.5370703199999998, 0.0541506, 2.4844599999999999, 113.71504, 92.431939999999997, 49.944319999999998, 60268.0, 10759.219999999999, 5.6834000000000003e+26] )
uranus = np.array( [19.191263930000002, 0.047167710000000002, 0.76985999999999999, 74.229879999999994, 170.96423999999999, 313.23218000000003, 25559.0, 30685.400000000001, 8.6813e+25] )
neptune = np.array( [30.068963480000001, 0.0085858700000000007, 1.7691699999999999, 131.72169, 44.971350000000001, 304.88002999999998, 24764.0, 60189.0, 1.0241299999999999e+26] )

In [168]:
#Create new column, considering G = 1
# Mass of the Sum, in kg
mass_sun_kg = 1988500e24

# Mass of the Sun, with G = 1
mass_sun_grav = 2.959139768995959e-04

# Conic section is ellipse
ialpha = -1

# Gravitational factor of the Sun
gm =  2.959139768995959e-04

In [171]:
# Create mass_grav column from mass = planet[8]
# Create gmpl column from mass_grav = planet[9]

for i in [8, 9]:
    mercury = np.append(mercury, mercury[i] * mass_sun_grav / mass_sun_kg)
    venus   = np.append(venus, venus[i] * mass_sun_grav / mass_sun_kg)
    earth   = np.append(earth, earth[i] * mass_sun_grav / mass_sun_kg)
    mars    = np.append(mars, mars[i] * mass_sun_grav / mass_sun_kg)
    jupiter = np.append(jupiter, jupiter[i] * mass_sun_grav / mass_sun_kg)
    saturn  = np.append(saturn, saturn[i] * mass_sun_grav / mass_sun_kg)
    uranus  = np.append(uranus, uranus[i] * mass_sun_grav / mass_sun_kg)
    neptune = np.append(neptune, neptune[i] * mass_sun_grav / mass_sun_kg)

In [184]:
row_planets = level1 * level2
columns_planets = len(mercury)

mercury2 = np.empty((columns_planets, row_planets), float)

for i in range(len(row_planets)):
    

SyntaxError: invalid syntax (<ipython-input-184-8852b9d46947>, line 6)

In [185]:
np.repeat(2,3)

array([2, 2, 2])

In [254]:
mercury = np.array( [0.38709893000000001, 0.20499999999999999, 7.0, 2439.6999999999998, 87.968999999999994, 3.3011000000000001e+23] )
venus = np.array( [0.72333199000000004, 0.0067732299999999999, 3.3947099999999999, 6051.8000000000002, 224.70099999999999, 4.8674999999999993e+24] )
earth = np.array( [1.00000011, 0.016710220000000001, 5.0000000000000002e-05, 6378.1369999999997, 365.25599999999997, 5.9722999999999996e+24] )
mars = np.array( [1.52366231, 0.093412330000000002, 1.8506100000000001, 3396.1999999999998, 686.98000000000002, 6.4171000000000003e+23] )
jupiter = np.array( [5.2033630100000003, 0.048392659999999997, 1.3052999999999999, 71492.0, 4332.5889999999999, 1.8981900000000001e+27] )
saturn = np.array( [9.5370703199999998, 0.0541506, 2.4844599999999999, 60268.0, 10759.219999999999, 5.6834000000000003e+26] )
uranus = np.array( [19.191263930000002, 0.047167710000000002, 0.76985999999999999, 25559.0, 30685.400000000001, 8.6813e+25] )
neptune = np.array( [30.068963480000001, 0.0085858700000000007, 1.7691699999999999, 24764.0, 60189.0, 1.0241299999999999e+26] )

In [255]:
row_planets = level1 * level2
columns_planets = len(mercury)

mercury = np.array([mercury,]*row_planets)
venus = np.array([venus,]*row_planets)
earth = np.array([earth,]*row_planets)
mars = np.array([mars,]*row_planets)
jupiter = np.array([jupiter,]*row_planets)
saturn = np.array([saturn,]*row_planets)
uranus = np.array([uranus,]*row_planets)
neptune = np.array([neptune,]*row_planets)

In [238]:
# Create mass_grav column from mass = planet[8]
# Create gmpl column from mass_grav = planet[9]

for i in [5, 6]:
    mercury = np.append(mercury, mercury[:,i] * mass_sun_grav / mass_sun_kg)
    venus   = np.append(venus, venus[:,i] * mass_sun_grav / mass_sun_kg)
    earth   = np.append(earth, earth[:,i] * mass_sun_grav / mass_sun_kg)
    mars    = np.append(mars, mars[:,i] * mass_sun_grav / mass_sun_kg)
    jupiter = np.append(jupiter, jupiter[:,i] * mass_sun_grav / mass_sun_kg)
    saturn  = np.append(saturn, saturn[:,i] * mass_sun_grav / mass_sun_kg)
    uranus  = np.append(uranus, uranus[:,i] * mass_sun_grav / mass_sun_kg)
    neptune = np.append(neptune, neptune[:,i] * mass_sun_grav / mass_sun_kg)

IndexError: too many indices for array

In [241]:
# Create mass_grav column from mass = planet[8]
# Create gmpl column from mass_grav = planet[9]

for i in [5, 6]:
    mercury[:,:-1] = mercury[:,i] * mass_sun_grav / mass_sun_kg
    venus[:,:-1]   = venus[:,i] * mass_sun_grav / mass_sun_kg
    earth[:,:-1]   = earth[:,i] * mass_sun_grav / mass_sun_kg
    mars[:,:-1]    = mars[:,i] * mass_sun_grav / mass_sun_kg
    jupiter[:,:-1] = jupiter[:,i] * mass_sun_grav / mass_sun_kg
    saturn[:,:-1]  = saturn[:,i] * mass_sun_grav / mass_sun_kg
    uranus[:,:-1]  = uranus[:,i] * mass_sun_grav / mass_sun_kg
    neptune[:,:-1] = neptune[:,i] * mass_sun_grav / mass_sun_kg

ValueError: could not broadcast input array from shape (20) into shape (20,5)

In [256]:
mercury = np.c_[mercury[:,5] * mass_sun_grav / mass_sun_kg]

In [257]:
mercury

array([[  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11],
       [  4.91245476e-11]])

In [251]:
N = 3
a = np.random.rand(N,N)
b = np.zeros((N,N+1))
b[:,:-1] = a

In [252]:
a

array([[ 0.11130408,  0.25218822,  0.60524686],
       [ 0.39446479,  0.38673055,  0.54294206],
       [ 0.78194495,  0.19033443,  0.75884607]])

In [253]:
b

array([[ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.]])

In [203]:
for i in planets_name:
    print(i)

mercury
venus
earth
mars
jupiter
saturn
uranus
neptune
